In [1]:
# IPython - biblioteka do wyświtlania podglądu oraz widżetów
import traitlets
import ipywidgets
from IPython.display import display

# Interfejs kamery.ipynb_checkpoints/i silników - JetRacer
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
from jetracer.nvidia_racecar import NvidiaRacecar

# Podstawowe biblioteki Python do przetwarzania obrazów
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [2]:
#tworzenie wykorzystywanych obiektów
camera = CSICamera(width=224, height=224, capture_device=0, capture_fps = 30)
car = NvidiaRacecar()
controller = ipywidgets.Controller(index=0)

#parametryzacja widżetów
widget_width = camera.width
widget_height = camera.height

image_widget = ipywidgets.Image(format='jpeg', width=widget_width, height=widget_height)
target_widget = ipywidgets.Image(format='jpeg', width=widget_width, height=widget_height)

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * widget_width / 2 + widget_width / 2)
    y = int(y * widget_height / 2 + widget_height / 2)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (int(widget_width / 2), widget_height), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (int(widget_width / 2), widget_height), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)


In [4]:
# zaprogramowanie kontrolera
traitlets.dlink((controller.axes[2], 'value'), (x_slider, 'value'), transform=lambda x: x_slider.value+x*0.1)
traitlets.dlink((controller.axes[3], 'value'), (y_slider, 'value'), transform=lambda x: y_slider.value+x*0.1)

left_right = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
forward = traitlets.dlink((controller.buttons[7], 'value'), (car, 'throttle'), transform=lambda x: -x)
backward = traitlets.dlink((controller.buttons[6], 'value'), (car, 'throttle'), transform=lambda x: x)

In [5]:
#tworzenie zbioru zdjęć z przypisanymi parametrami
DATASET_DIR = 'ad_dataset_sala230_as'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

count_widget = ipywidgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x*widget_width/2+widget_width/2, y*widget_height/2+widget_height/2, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
# camera_widget.on_msg(save_snapshot)        

controller.buttons[0].observe(save_snapshot, names='value')

In [6]:
#wyświetlanie widżetów
display(ipywidgets.HBox([image_widget, target_widget]), x_slider, y_slider)
display(ipywidgets.VBox([
    count_widget,
    controller    
]))

camera.running = True

FloatSlider(value=-0.01284046769142151, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.005369651317596436, description='y', max=1.0, min=-1.0, step=0.001)

In [ ]:
camera.running = False